<a href="https://colab.research.google.com/github/ad451/Stackoverflow_tag_generator/blob/main/StackOverflow_Tag_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importing the required modules**

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
from bs4 import BeautifulSoup
import time
import warnings; warnings.simplefilter('ignore')
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('averaged_perceptron_tagger') # required for parts of speech
nltk.download('wordnet') # required for parts of speech
nltk.download('stopwords') #download the stopwords


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Web Scraping the current set of questions for testing the model**

In [ ]:


Questions=[] #array to store the questions

for pageNumber in range(1,401):
    response=requests.get("https://stackoverflow.com/questions",params={"tab":"newest","page":pageNumber,"pagesize":50})

    data=BeautifulSoup(response.text,'html.parser' ) #parsing the html text

    req_data=data.find(id="questions")

    new_data=req_data.find_all("h3", class_="s-post-summary--content-title") # the tag that contains the question info



    for element in new_data:
        link=element.a.attrs['href']

        response=requests.get(f"https://stackoverflow.com/{link}")  #fetching the content related to the question

        data_questionwise=BeautifulSoup(response.text,'html.parser')

        question_wise_title=data_questionwise.find("div",id="question-header").h1.a.string #title

        question_wise_desc=data_questionwise.find("div",class_="s-prose js-post-body") #description

        all_paragraphs=question_wise_desc.find_all("p")

        total_description_question_wise=""

        for para in all_paragraphs:
            total_description_question_wise+=para.text

        Final_content=question_wise_title+""+total_description_question_wise  #concatenating the title and description

        tag_question_wise=data_questionwise.find("ul",class_="ml0 list-ls-none js-post-tag-list-wrapper d-inline").li.text #tag

        Questions.append([Final_content,tag_question_wise])

    print(pageNumber)   #checking which page questions have been fetched yet

print(len(Questions))








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.2 MB/s eta 0:00:00


**Sql code to combine the input questions and tags table**

In [ ]:
###################################### Code for inner combine ######################################
df1 = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('Tags.csv', encoding='ISO-8859-1')

# combined dataframe of questiontags
df3 = df1.set_index('Id').join(df2.set_index('Id'))
df3 = df3.reset_index()
df4 = pd.DataFrame(df3.Id.value_counts())
df4 = df4.reset_index()

# df5 is the final combined dataframe of questiontags. Randomly picked datapoints from df3
store = {}
df5 = pd.DataFrame()
n_train = 20000
for i in range(n_train):
    key = 0
    while(key==0):
        a = random.randint(df4.shape[0])
        if a in store:
            continue
        else:
            key==1
            store[a] = 1
            ind = df3[df3.Id==df4.iloc[a,0]].first_valid_index()
            df5 =  pd.concat([df5,pd.DataFrame(df3.iloc[ind]).T],ignore_index=True)
            break

###################################### Code for removing angular brackets from the "Body" column ######################################

for j in range(df5.shape[0]):
    ans=" "
    curr = 0
    flag = 0
    while(curr<len(df5['Body'][j])):
        if( df5['Body'][j][curr]=='<'):
            flag = 1
            curr = curr + 1
            continue
        if( df5['Body'][j][curr]=='>'):
            flag = 0
            curr = curr + 1
            continue
        if(flag == 1):
            curr = curr + 1
        if(flag == 0):
            ans = ans + df5['Body'][j][curr]
            curr = curr + 1
    df1['Title'][j] += ans

df6 = pd.concat([df5.iloc[:,6], df5.iloc[:,8]], axis=1)

**Machine learning part (Preprocessing and training)**

---



In [5]:
train_data=pd.read_csv("questiontags_train.csv")
test_data=pd.read_csv("questiontags_test.csv")

In [6]:
#rename the columns of the train dataset

train_data.drop(train_data.columns[0],axis=1,inplace=True)


train_data.rename(columns={"Title":"Questions","Tag":"Tags"},inplace=True)


In [7]:
import random
#code to include around 10000 more questions to the train dataset from the test dataset
v={}
count=0
while count!=10000:
    idx=random.randint(0,test_data.shape[0]-1)
    if idx not in v:
      train_data=train_data.append(test_data.loc[idx],ignore_index=True)
      test_data.drop(idx,inplace=True,axis=0)
      count+=1
      v[idx]=1
    else:
      continue


In [8]:
train_data['Questions']=train_data['Questions'].str.replace('</p>',' ')
train_data['Questions']=train_data['Questions'].str.replace('\n',' ')
train_data['Questions']=train_data['Questions'].str.replace('</a>',' ')
train_data['Questions']=train_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls
train_data['Questions'] = train_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) #removes small length words (len<3)
train_data['Questions']=train_data['Questions'].str.replace('<a href=" ">','')

train_data['Questions']=train_data['Questions'].apply(lambda x : x.lower()) #coverting to lowercase


In [9]:
test_data['Questions']=test_data['Questions'].str.replace('</p>',' ')
test_data['Questions']=test_data['Questions'].str.replace('\n',' ')
test_data['Questions']=test_data['Questions'].str.replace('</a>',' ')
test_data['Questions'] = test_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
test_data['Questions']=test_data['Questions'].str.replace('<a href=" ">','')
test_data['Questions']=test_data['Questions'].apply(lambda x : x.lower())
test_data['Questions']=test_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls

In [10]:
#removing the stop words and the punctuations from test and train dataset


stop_words=set(nltk.corpus.stopwords.words('english'))
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#removing the punctuations from test and train dataset


punctuations = string.punctuation
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))



In [12]:
test_data=test_data.reset_index()


In [13]:

test_data.drop(['index'],inplace=True,axis=1)

In [14]:
#dropping the rows with empty values of Question after filtering
for j in range(len(test_data['Questions'])):
  if len(test_data['Questions'][j])==0:
     test_data.drop(j,inplace=True)

for j in range(len(train_data['Questions'])):
  if len(train_data['Questions'][j])==0:
     train_data.drop(j,inplace=True)
     print("yes")

yes
yes
yes
yes
yes
yes


In [15]:
test_data=test_data.reset_index()
train_data=train_data.reset_index()

In [16]:
test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)


In [71]:
#using lemmatization on the questions of the train and test dataset
def lemmatization(text):
    pos_dict = {
        'N': 'n',  # Noun
        'V': 'v',  # Verb
        'R': 'r',  # Adverb
        'J': 'a'   # Adjective
    }
    pos_tags = pos_tag(text)
    lemmatizer = WordNetLemmatizer()
    lemma=[]
    for word, tag in pos_tags:
        if (tag[0].upper() not in pos_dict.keys()):
          pos='n'
        else:
          pos= pos_dict[tag[0].upper()]
        lemma.append(lemmatizer.lemmatize(word,pos=pos))
    return lemma

train_data['Questions']=train_data['Questions'].apply(lambda x : lemmatization(x.split()))
test_data['Questions']=test_data['Questions'].apply(lambda x : lemmatization(x.split()))


**Data Exploration with dev of Base Model**

In [20]:
questions = train_data['Questions'].tolist()
tags = train_data['Tags'].tolist()

print('The total number of words in the data is: ', sum([len(text.split()) for text in questions]))

def tokenize_question(text):
    return text.split()

question_vect = CountVectorizer(tokenizer=tokenize_question)
questions=question_vect.fit_transform(questions)

print('The number of words in the vocabulary is: ', len(question_vect.vocabulary_))

def tokenize_tags(text):
    return text.split('|')

max_tags = 100
tags_vect = CountVectorizer(tokenizer=tokenize_tags, max_features=max_tags)
tags = tags_vect.fit_transform(tags)
tags = tags.toarray()
print('Number of tags: ', len(tags_vect.vocabulary_))

tags_token = tags_vect.get_feature_names_out()
tag_frequency = tags.sum(axis=0)
print('The list of tags with frequency is: ')
print(dict(zip(tags_token, tag_frequency)))



The total number of words in the data is:  2192542
The number of words in the vocabulary is:  272590
Number of tags:  100
The list of tags with frequency is: 
{'.htaccess': 30, '.net': 153, 'actionscript-3': 39, 'ajax': 42, 'algorithm': 66, 'amazon-web-services': 128, 'android': 1403, 'angular': 115, 'angularjs': 164, 'apache': 53, 'apache-spark': 32, 'arrays': 51, 'asp.net': 223, 'asp.net-mvc': 100, 'assembly': 28, 'azure': 127, 'bash': 88, 'batch-file': 27, 'c': 403, 'c#': 2013, 'c++': 968, 'cakephp': 28, 'css': 259, 'database': 85, 'delphi': 55, 'django': 150, 'docker': 112, 'eclipse': 64, 'elasticsearch': 40, 'excel': 212, 'facebook': 54, 'firebase': 35, 'flutter': 202, 'git': 147, 'go': 67, 'google-apps-script': 42, 'hadoop': 43, 'haskell': 35, 'html': 480, 'html5': 39, 'image': 26, 'ios': 681, 'iphone': 279, 'java': 2339, 'javascript': 2850, 'jquery': 532, 'jsf': 27, 'json': 86, 'kubernetes': 48, 'laravel': 58, 'linux': 195, 'lua': 28, 'machine-learning': 29, 'magento': 28, 'matl

In [21]:
tag_count = [[x[0],x[1]] for x in zip(tags_token, tag_frequency)]
tag_count = sorted(tag_count, key = lambda x: x[1], reverse=True)

print('Rank     Tag      Count')
for i in range(100):
    print("%d   %s   %d" % (i+1, tag_count[i][0], tag_count[i][1]))

Rank     Tag      Count
1   javascript   2850
2   java   2339
3   python   2254
4   c#   2013
5   php   1714
6   android   1403
7   c++   968
8   ios   681
9   jquery   532
10   r   497
11   html   480
12   sql   477
13   c   403
14   ruby-on-rails   376
15   mysql   357
16   node.js   336
17   reactjs   320
18   iphone   279
19   css   259
20   asp.net   223
21   excel   212
22   flutter   202
23   linux   195
24   objective-c   171
25   angularjs   164
26   .net   153
27   django   150
28   windows   148
29   git   147
30   ruby   143
31   sql-server   141
32   amazon-web-services   128
33   azure   127
34   swift   122
35   angular   115
36   python-3.x   113
37   docker   112
38   typescript   107
39   regex   103
40   asp.net-mvc   100
41   xml   95
42   vb.net   94
43   wordpress   92
44   postgresql   91
45   matlab   90
46   bash   88
47   json   86
48   database   85
49   wpf   82
50   scala   81
51   spring   75
52   powershell   73
53   mongodb   71
54   go   67
55   algorit

In [28]:
# X_test = test_data['Questions'].tolist()
# y_test = test_data['Tags'].tolist()

In [29]:
# X_train, X_test, y_train, y_test = train_test_split(questions, tags,
#                                                     test_size=0.2,
#                                                     random_state=747)

# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
#                                                     test_size=0.5,
#                                                     random_state=747)

In [30]:
def eval_metrics(y_test, y_predicted, print_metrics=True):

    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')

    if print_metrics:
        print("f1: %.3f - precision: %.3f - recall: %.3f - accuracy: %.3f" % (
            f1, precision, recall, accuracy))
    return f1, precision, recall, accuracy

AttributeError: ignored

In [49]:
# def tokenize_question(text):
#     return text.split()

tfidf_vect = TfidfVectorizer(tokenizer=tokenize_question,
                               stop_words='english',
                               min_df=2,
                               max_df=0.5)

X_train_tfidf = tfidf_vect.fit_transform(train_data["Questions"])
X_test_tfidf = tfidf_vect.transform(test_data["Questions"])
print('The number of words in the vocabulary is: ', len(tfidf_vect.vocabulary_))

feature_names = tfidf_vect.get_feature_names_out()

# for i, feature in enumerate(feature_names):
#     print("Feature:", feature)
#     print("Train TF-IDF:", max(X_train_tfidf[:, i]))
#     if i==20:
#       break



The number of words in the vocabulary is:  47064
47064


In [ ]:
%%time
tfidf_log_clf = OneVsRestClassifier(LogisticRegression())

tfidf_log_clf.fit(X_train_tfidf, y_train)

CPU times: user 27.6 s, sys: 28.5 s, total: 56.1 s
Wall time: 32 s


OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
y_train_tfidf_predict = tfidf_log_clf.predict(X_train_tfidf)
eval_metrics(y_train, y_train_tfidf_predict)

f1: 0.287 - precision: 0.705 - recall: 0.192 - accuracy: 0.325


(0.2869979496204687,
 0.7048346764490622,
 0.19239736441966548,
 0.3247759949989581)

In [ ]:
y_test_tfidf_predict = tfidf_log_clf.predict(X_test_tfidf)
eval_metrics(y_test, y_test_tfidf_predict)

f1: 0.246 - precision: 0.563 - recall: 0.164 - accuracy: 0.287


(0.24568413711051668, 0.5629343082208758, 0.1643342662934826, 0.287)